## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [31]:
# 1. Import the WeatherPy_Database.csv file.
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tuatapere,NZ,-46.1333,167.6833,48.00,80,4,8.01,clear sky
1,1,Mataura,NZ,-46.1927,168.8643,48.00,80,30,8.01,scattered clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.64,76,3,18.92,clear sky
3,3,Jamestown,US,42.0970,-79.2353,21.00,73,90,16.11,light snow
4,4,Khatanga,RU,71.9667,102.5000,-45.09,73,0,2.08,clear sky


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Atuona,PF,-9.8000,-139.0333,78.64,76,3,18.92,clear sky
6,6,Sao Filipe,CV,14.8961,-24.4956,75.13,63,84,9.60,broken clouds
8,8,Kapaa,US,22.0752,-159.3190,73.99,78,75,20.71,broken clouds
9,9,Tadine,NC,-21.5500,167.8833,76.93,79,54,16.62,broken clouds
10,10,Bambous Virieux,MU,-20.3428,57.7575,81.00,78,75,9.22,light rain
11,11,Raga,SS,8.4596,25.6780,77.88,16,54,8.12,broken clouds
15,15,Puerto Suarez,BO,-18.9500,-57.8000,89.60,74,75,11.50,broken clouds
16,16,Isangel,VU,-19.5500,169.2667,75.20,88,20,14.36,few clouds
18,18,Arraial Do Cabo,BR,-22.9661,-42.0278,89.60,65,0,19.57,clear sky
19,19,Uchiza,PE,-8.4592,-76.4633,84.69,62,42,2.30,scattered clouds


In [34]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                294
City                   294
Country                291
Lat                    294
Lng                    294
Max Temp               294
Humidity               294
Cloudiness             294
Wind Speed             294
Current Description    294
dtype: int64

In [35]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [36]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.64,clear sky,-9.8000,-139.0333,
6,Sao Filipe,CV,75.13,broken clouds,14.8961,-24.4956,
8,Kapaa,US,73.99,broken clouds,22.0752,-159.3190,
9,Tadine,NC,76.93,broken clouds,-21.5500,167.8833,
10,Bambous Virieux,MU,81.00,light rain,-20.3428,57.7575,
11,Raga,SS,77.88,broken clouds,8.4596,25.6780,
15,Puerto Suarez,BO,89.60,broken clouds,-18.9500,-57.8000,
16,Isangel,VU,75.20,few clouds,-19.5500,169.2667,
18,Arraial Do Cabo,BR,89.60,clear sky,-22.9661,-42.0278,
19,Uchiza,PE,84.69,scattered clouds,-8.4592,-76.4633,


In [37]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [38]:
# 7. Drop the rows where there is no Hotel Name.
#clean_hotel_df = hotel_df.dropna()
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.64,clear sky,-9.8000,-139.0333,Villa Enata
6,Sao Filipe,CV,75.13,broken clouds,14.8961,-24.4956,Tortuga B&B
8,Kapaa,US,73.99,broken clouds,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
10,Bambous Virieux,MU,81.00,light rain,-20.3428,57.7575,Casa Tia Villa
15,Puerto Suarez,BO,89.60,broken clouds,-18.9500,-57.8000,Hotel casa real
...,...,...,...,...,...,...,...
681,Mangochi,MW,74.84,light rain,-14.4782,35.2645,Fort Johnstone Hotel
683,Galdar,ES,71.01,clear sky,28.1470,-15.6502,Hostel Guide
686,Pitimbu,BR,82.40,broken clouds,-7.4706,-34.8086,Reserva do Abiaí
688,Banikoara,BJ,79.32,clear sky,11.2985,2.4386,Hôtel de l'UCom CVPC


In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [40]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

"""
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and <dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))